In [1]:
import pandas as pd
import numpy as np

In [146]:
df = pd.read_csv('clean_data.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [147]:
df['tweet_text'] = df['tweet_text'].astype(str)

In [148]:
df['emotion_in_tweet_is_directed_at'].value_counts()

Apple      5344
Google     2435
none        815
Android     499
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [149]:
df = df[df['emotion_in_tweet_is_directed_at'] != "none"]


In [150]:
android_join  = df[df['emotion_in_tweet_is_directed_at'].str.contains("Android")]

android_join['emotion_in_tweet_is_directed_at'].replace({"Android": 'Google'}, inplace=True)

df = android_join.combine_first(df)

In [151]:
df.drop('emotion', axis=1, inplace=True)

In [152]:
df['emotion_in_tweet_is_directed_at'].value_counts()

Apple     5344
Google    2934
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [153]:
import re
# function to clean text
def cleaner(tweet):
    tweet = re.sub(r'@[A-Za-z0-9]+','', str(tweet)) # removes @
    tweet = re.sub(r'#', '', str(tweet)) # removes hashtag
    tweet = re.sub(r'RT[\s]+','', str(tweet)) # removes RT
    tweet = re.sub(r'https?:\/\/\S+', '', tweet) # remove hyperlink in tweet
    tweet = re.sub(r'[^\w\s]', '', tweet) # removes punctuations
    #tweet = re.sub(r'[^a-zA-Z]','', tweet)
    return tweet

df['tweet_text'] = df['tweet_text'].apply(cleaner)

In [154]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


tokenizer = RegexpTokenizer(r"(?u)\w{3,}[a-zA-Z]")

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()


In [155]:
def preprocess_text(text, tokenizer, stopwords_list, lemmatizer):
    lowered = text.lower()
    tokens = tokenizer.tokenize(lowered)
    stopped_tokens = [word for word in tokens if word not in stopwords_list]
    lemma = [lemmatizer.lemmatize(token) for token in stopped_tokens]
    
    return lemma

    preprocessed = pd.concat(tokens, stopped_tokens, lemma)
    


In [156]:
text_data = df.tweet_text.apply(lambda x: preprocess_text(x, tokenizer, stopwords_list, lemmatizer))
text_data

0       [iphone, tweeting, rise_austin, dead, need, up...
1       [know, awesome, ipadiphone, youll, likely, app...
2                          [wait, ipad, also, sale, sxsw]
3       [hope, year, festival, isnt, crashy, year, iph...
4       [great, stuff, sxsw, marissa, mayer, google, o...
                              ...                        
9088                       [ipad, everywhere, sxsw, link]
9089    [wave, buzz, interrupt, regularly, scheduled, ...
9090    [google, zeiger, physician, never, reported, p...
9091    [verizon, iphone, customer, complained, time, ...
9092    [ïïàü_êîòáââ_â_ûâgoogle, test, ûïcheckin, offe...
Name: tweet_text, Length: 8278, dtype: object

In [157]:
df['tweet_text'] = text_data
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,"[iphone, tweeting, rise_austin, dead, need, up...",Apple,Negative emotion
1,"[know, awesome, ipadiphone, youll, likely, app...",Apple,Positive emotion
2,"[wait, ipad, also, sale, sxsw]",Apple,Positive emotion
3,"[hope, year, festival, isnt, crashy, year, iph...",Apple,Negative emotion
4,"[great, stuff, sxsw, marissa, mayer, google, o...",Google,Positive emotion
...,...,...,...
9088,"[ipad, everywhere, sxsw, link]",Apple,Positive emotion
9089,"[wave, buzz, interrupt, regularly, scheduled, ...",Google,No emotion toward brand or product
9090,"[google, zeiger, physician, never, reported, p...",Google,No emotion toward brand or product
9091,"[verizon, iphone, customer, complained, time, ...",Apple,No emotion toward brand or product
